In [6]:
from pymongo import MongoClient

client = MongoClient(
    "mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set"
)

db = client.arxiv_db

# 1. Conteo total
total = db.articles.estimated_document_count()
print(f"📊 Total documentos: {total:,}")

# 2. Verificar consistencia entre nodos
for member in client.admin.command("replSetGetStatus")["members"]:
    print(f"🔹 Nodo {member['name']}: {'✅ HEALTHY' if member['health'] == 1 else '❌ PROBLEM'}")

# 3. Verificar muestras por categoría
print("\n📝 Muestra de documentos cs.AI:")
for doc in db.articles.find({"categories": "cs.AI"}, {"_id": 0, "title": 1}).limit(3):
    print(f" - {doc['title']}")

# 4. Verificar campos importantes
print("\n🔍 Verificación de campos críticos:")
stats = {
    "Con DOI": db.articles.count_documents({"doi": {"$exists": True}}),
    "Con PDF": db.articles.count_documents({"pdf_source": {"$exists": True}}),
    "Con autores": db.articles.count_documents({"authors": {"$exists": True, "$ne": []}})
}
# for k, v in stats.items():
#     print(f" - {k}: {v:,} ({v/total:.1%})")

📊 Total documentos: 550,984
🔹 Nodo mongo1:30001: ✅ HEALTHY
🔹 Nodo mongo2:30002: ✅ HEALTHY
🔹 Nodo mongo3:30003: ✅ HEALTHY

📝 Muestra de documentos cs.AI:
 - Calculating Valid Domains for BDD-Based Interactive Configuration
 - A study of structural properties on profiles HMMs
 - Bayesian approach to rough set

🔍 Verificación de campos críticos:


In [2]:
db.drop_collection('articles')

{'nIndexesWas': 1,
 'ns': 'arxiv_db.articles',
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1749588072, 1),
  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'keyId': 0}},
 'operationTime': Timestamp(1749588072, 1)}